# Sprint 3 - Dual Transformation and Non-Smooth Utility

This notebook focuses on the PRD core novelty:
- kinked/non-smooth utility
- convex conjugate in dual space
- dual PDE solve and primal recovery
- boundary shift under non-smooth incentives

**Roadmap:** Sprint 3 (Weeks 5-6).

In [ ]:
import numpy as np

from optistop import (
    GBM,
    Grid,
    KinkedUtility,
    PSORSolver,
    PrimalObstacleSolver,
    DualTransformationSolver,
)
from optistop.solvers.fdm import FDMOperator

In [ ]:
# Non-smooth utility: piecewise slopes around incentive thresholds
utility = KinkedUtility(
    thresholds=(1.2, 2.0),
    slopes=(0.0, 0.2, 0.08),
    intercept=0.0,
)

x_probe = np.linspace(0.0, 3.0, 10)
print('Utility values:', utility.evaluate(x_probe))
print('Utility derivative (piecewise):', utility.evaluate_derivative(x_probe))
print('Kink points:', utility.kink_points())

In [ ]:
# Primal solve (obstacle in x-domain)
r, mu, sigma = 0.05, 0.015, 0.30
process = GBM(mu=mu, sigma=sigma)
operator = FDMOperator(process=process, r=r)
psor = PSORSolver(omega=1.25, tol=1e-6, max_iter=40_000)

x_grid = Grid(x_min=1e-4, x_max=6.0, n=2000)
primal_result = PrimalObstacleSolver(grid=x_grid, utility=utility, operator=operator, psor=psor).solve()

print('Primal trigger x* =', primal_result.trigger)
print('Primal method =', primal_result.metadata.get('method'))

In [ ]:
# Dual solve (linearized via transform in y-domain)
y_grid = Grid(x_min=1e-3, x_max=6.0, n=1500)
dual_operator = FDMOperator(process=GBM(mu=mu, sigma=sigma), r=r)

# Note: x_bounds_for_conjugate should cover practical wealth domain.
dual_solver = DualTransformationSolver(
    primal_grid=x_grid,
    dual_grid=y_grid,
    utility=utility,
    dual_operator=dual_operator,
    x_bounds_for_conjugate=(0.0, 8.0),
)

dual_result = dual_solver.solve()
print('Dual-recovered trigger x* =', dual_result.trigger)
print('Dual method =', dual_result.metadata.get('method'))

In [ ]:
# Boundary shift interpretation for manager incentives
print('Trigger difference (dual - primal):', dual_result.trigger - primal_result.trigger)

if dual_result.trigger > primal_result.trigger:
    print('Interpretation: stronger wait-to-invest tendency under transformed valuation.')
else:
    print('Interpretation: earlier exercise tendency under current utility specification.')

## Sprint 3 Deliverable Check

- [x] Kinked utility class instantiated and inspected
- [x] Primal non-smooth solve completed
- [x] Dual transform solve completed with primal recovery
- [x] Boundary shift compared for interpretation